Imports

In [ ]:
from bs4 import BeautifulSoup, Tag
import requests
import pprint
!pip install stanza
import re

In [ ]:
import pandas as pd
import stanza

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Thesis2023/philoalex-main.zip

In [ ]:
import glob
import pandas as pd
grc_files = sorted(glob.glob('/content/drive/MyDrive/Thesis2023/XML Files/Yonge_Books/*.opp-grc1.xml'))
grc_files
#grc_colson=

['/content/drive/MyDrive/Thesis2023/XML Files/Yonge_Books/tlg0018.tlg002.opp-grc1.xml',
 '/content/drive/MyDrive/Thesis2023/XML Files/Yonge_Books/tlg0018.tlg019.opp-grc1.xml',
 '/content/drive/MyDrive/Thesis2023/XML Files/Yonge_Books/tlg0018.tlg022.opp-grc1.xml',
 '/content/drive/MyDrive/Thesis2023/XML Files/Yonge_Books/tlg0018.tlg024.opp-grc1.xml']

In [ ]:
#use whichever language you need
#nlp_grc = stanza.Pipeline(lang='grc', processors='tokenize')
nlp = stanza.Pipeline(lang='grc', processors='tokenize')

In [ ]:
philo = open( '/content/corr-tlg0018.tlg020.opp-grc1.xml' ,encoding='utf-8', errors='ignore')
philo_soup = BeautifulSoup(philo,'lxml')

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [ ]:
philo_soup

In [ ]:
sections= philo_soup.find_all('div', subtype= 'section')
for sec in sections:
  sec['id']= 'Section',sec['n']


In [ ]:
notes= philo_soup.find_all("note")
for note in notes:
  note.decompose()
    #removing marked mistakes from Prof Crane's corrected file (<sic> tag)
sic= philo_soup.find_all('sic')
for s in sic:
  s.decompose()
    #removing page break tags
pages = philo_soup.find_all('pb')
for p in pages:
  p.decompose()
    #removing line break tags
breaks = philo_soup.find_all('lb')
for b in breaks:
  b.decompose()


In [ ]:
for sec in philo_soup.find_all('div', subtype= 'section'):
  for p in sec.find_all('p'):
    p['id']= sec['id']

In [ ]:
philo_soup

In [ ]:
paras = philo_soup.find_all('p')
for para in paras:
  doc= nlp(para.get_text())   #applies tokenizer
  sents= [sentence.text for sentence in doc.sentences]  #acceses tuple that stanza makes of sentences
  para.clear()  #not sure why this needs to be in here to work but...it does
  for sent_index, sent in enumerate(sents): #enumersates the sentences for each section
    sent_tag = philo_soup.new_tag('s',n= sent_index+1)
    sent_tag.string = sent #identifies contents of the tag
    sec.append(sent_tag) #appends contents to the correct tag



XML Edits

In [ ]:
#loading the file with beautiful soup
def open_file(x):
  philo = open( x ,encoding='utf-8', errors='ignore')
  philo_soup = BeautifulSoup(philo,'lxml')
  return philo_soup



In [ ]:
#removing extra tags that we don't need for text tokenizing
def removing_tags(philo_soup):
  notes= philo_soup.find_all("note")
  for note in notes:
    note.decompose()
    #removing marked mistakes from Prof Crane's corrected file (<sic> tag)
  sic= philo_soup.find_all('sic')
  for s in sic:
    s.decompose()
    #removing page break tags
  pages = philo_soup.find_all('pb')
  for p in pages:
    p.decompose()
    #removing line break tags
  breaks = philo_soup.find_all('lb')
  for b in breaks:
    b.decompose()
  return philo_soup


In [ ]:
#alternate
def making_books_and_secs_II(philo_soup):
  for b in philo_soup.find_all('div', subtype= 'book'):
    b['id']= 'Book'+(str(b['n']))
    for sec in b.find_all('div', subtype= 'section'):
      sec['id']= b['id']+ 'Section'+(str(sec['n']))
      for para in sec.find_all('p'):
        para['id']= sec['id']
  return philo_soup

In [ ]:
#alternate
#not all indexes are aligned correctly with the section number
def make_sections(philo_soup):
  sections= philo_soup.find_all('div',subtype= 'section')
  paras= philo_soup.find_all('p')
  for sec in sections:
    sec['id']= 'Section'+ (str(sec['n']))
    for para in paras:
      para['id']= sec['id']
  return philo_soup


In [ ]:
#alternate
def sentence_tokenization(philo_soup):
  sections= philo_soup.find_all('div',subtype= 'section')
  for sec in sections:
    doc= nlp(sec.get_text())   #applies tokenizer
    sents= [sentence.text for sentence in doc.sentences]  #acceses tuple that stanza makes of sentences
    sec.clear()  #not sure why this needs to be in here to work but...it does
    for sent_index, sent in enumerate(sents): #enumersates the sentences for each section
      sent_tag = philo_soup.new_tag('s', n= sent_index+1)
      sent_tag.string = sent #identifies contents of the tag
      sec.append(sent_tag) #appends contents to the correct tag
  return philo_soup


In [ ]:
#alternate
def making_sent_tags_II(philo_soup):
  for sec in philo_soup.find_all('div',subtype= 'section'):
    for s in sec.find_all('s'):
      s['id']= sec['id']+'Sent'+(str(s['n']))
  return philo_soup

In [ ]:
def books_and_sections(philo_soup):
  books= philo_soup.find_all('div', subtype= 'book')
  for b_index, book in enumerate(books):
    book['id']= f'Book{b_index}'
    sections= philo_soup.find_all('p')
    for sec_index, sec in enumerate(sections):
      doc= nlp(sec.get_text())   #applies tokenizer
      sents= [sentence.text for sentence in doc.sentences]  #acceses tuple that stanza makes of sentences
      sec.clear()  #not sure why this needs to be in here to work but...it does
      for sent_index, sent in enumerate(sents): #enumersates the sentences for each section
        sent_tag = philo_soup.new_tag('s',id= f'Book{b_index-1}section{sec_index}sent{sent_index+1}')
        sent_tag.string = sent #identifies contents of the tag
        sec.append(sent_tag) #appends contents to the correct tag
  return philo_soup


In [ ]:
#numbers sections based on index
#works only if sections numbers and index are aligned
#otherwise use s[id]= section, p['n]
def making_sections(philo_soup):
  sections= philo_soup.find_all('p')
  for index, s in enumerate(sections):
    s['id']= f'Section{index}'

  return philo_soup

In [ ]:
def making_sent_tags(philo_soup):
  sections= philo_soup.find_all('p')
  for sec_index, sec in enumerate(sections):
    doc= nlp(sec.get_text())   #applies tokenizer
    sents= [sentence.text for sentence in doc.sentences]  #acceses tuple that stanza makes of sentences
    sec.clear()  #not sure why this needs to be in here to work but...it does
    for sent_index, sent in enumerate(sents): #enumersates the sentences for each section
      sent_tag = philo_soup.new_tag('s',id= f'section{sec_index}sent{sent_index+1}')
      sent_tag.string = sent #identifies contents of the tag
      sec.append(sent_tag) #appends contents to the correct tag
  return philo_soup


In [ ]:
def write_file(philo_soup,x):
  with open(f"{x}_edited", 'w') as philo_file:
    philo_file.write(str(philo_soup))


In [ ]:
for x in grc_files:
  opened= open_file(x)
  removed = removing_tags(opened)
  sected= making_books_and_secs_II(removed)
  tokenized= sentence_tokenization(sected)
  sented= making_sent_tags_II(tokenized)
  written= write_file(sented,x)


/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [ ]:
corr_files = sorted(glob.glob('/content/drive/MyDrive/Thesis2023/XML Files/Yonge_Books/*-grc1.xml_edited'))
corr_files

['/content/drive/MyDrive/Thesis2023/XML Files/Yonge_Books/tlg0018.tlg002.opp-grc1.xml_edited',
 '/content/drive/MyDrive/Thesis2023/XML Files/Yonge_Books/tlg0018.tlg019.opp-grc1.xml_edited',
 '/content/drive/MyDrive/Thesis2023/XML Files/Yonge_Books/tlg0018.tlg022.opp-grc1.xml_edited',
 '/content/drive/MyDrive/Thesis2023/XML Files/Yonge_Books/tlg0018.tlg024.opp-grc1.xml_edited']

In [ ]:
import zipfile

In [ ]:

with zipfile.ZipFile('cohn_corr__books_grc_.zip', 'w') as f:
    for y in corr_files:
        f.write(y)

In [ ]:
#used this workflow/ code for the ones with books and sections
#not the most efficient but moves down the tree successively adding more info
for sec in philo_soup.find_all(subtype= 'section'):
  for para in sec.find_all('p'):
    para['id']= sec['id']
    for sent in para.find_all('s'):
      sent['id']= para['id']+'sent'+(str(sent['n']))